In [1]:
import collections
import networkx as nx
import sys
import os
sys.path.append(r'c:\\Users\\rotim\\OneDrive\\Documents\\Reading\\graph-code\\Skoltech-PhD-Thesis\\_codes\\SIR\\Influential-nodes-identification-in-complex-networks-via-information-entropy')
from algorithms import *
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from tqdm import tqdm
import pickle

In [2]:
graph_collections, graph_name = read_graph("./test-graphs")

{'ABILENE': {'index': 0, 'nodes': 11, 'edges': 14}, 'ABVT': {'index': 1, 'nodes': 23, 'edges': 31}, 'AGIS': {'index': 2, 'nodes': 25, 'edges': 30}, 'ARN.GRAPHML': {'index': 3, 'nodes': 30, 'edges': 29}, 'BELLCANADA': {'index': 4, 'nodes': 48, 'edges': 64}, 'BELLSOUTH.GRAPHML': {'index': 5, 'nodes': 51, 'edges': 66}, 'BICS.GRAPHML': {'index': 6, 'nodes': 33, 'edges': 48}, 'CERNET.GRAPHML': {'index': 7, 'nodes': 41, 'edges': 58}, 'CESNET200706.GRAPHML': {'index': 8, 'nodes': 44, 'edges': 51}, 'CHINANET.GRAPHML': {'index': 9, 'nodes': 42, 'edges': 66}, 'COGENTCO.GRAPHML': {'index': 10, 'nodes': 197, 'edges': 243}, 'COLT.GRAPHML': {'index': 11, 'nodes': 153, 'edges': 177}, 'DELTACOM.GRAPHML': {'index': 12, 'nodes': 113, 'edges': 161}, 'GARR201110.GRAPHML': {'index': 13, 'nodes': 59, 'edges': 74}, 'GEANT2001': {'index': 14, 'nodes': 27, 'edges': 38}, 'GRNET.GRAPHML': {'index': 15, 'nodes': 37, 'edges': 42}, 'INTELLIFIBER.GRAPHML': {'index': 16, 'nodes': 73, 'edges': 95}, 'INTEROUTE.GRAPHML'

In [3]:
G = graph_collections[1]
# geo_loc_data = get_geo_data("Internet2LatLong.csv")
geo_loc_data = {k: {"lat": v.get('Latitude', 0), "long": v.get('Longitude', 0)}
                for (k, v) in G.nodes(data=True)}
# print(geo_loc_data)
edge_geo_data_combined = assign_location(G, geo_loc_data)
# set edge attributes
set_edge_attr(G, edge_geo_data_combined)
# G.edges(data=True)


In [4]:
selection__=[0,2,4,23]# [37,41,20,0,14,4,1,2,10, 18, 8]

gGrahps = selection__from_graph(graph_collections, graph_name)  # , selection__

# print(gGrahps)
# assign coordinates: Lat and Long
geo_loc_data_sample = {i: {k: {"lat": v.get('Latitude', 0), "long": v.get('Longitude', 0)}
     for (k, v) in g.nodes(data=True)} for i, g in gGrahps}
# print(geo_loc_data_sample)

edge_geo_data_sample = {k: assign_location(u, v) for (k, v), (j,u) in zip(geo_loc_data_sample.items(), gGrahps)}
# print(geo_loc_data_sample)
nx.get_edge_attributes(gGrahps[4][1], 'weight')


{}

In [5]:
# Obtain node strength to the desired degree: 1,2,3,...
node_strength_2_deg = [(i, sum(hub_information(G, i, 2))) 
                       for (i, j) in sorted(list(G.degree()), key=lambda item: int(item[0]))]
degree__ = [(i, len(n_neighbor(G, i, 1))) for (i, j) in sorted(list(G.degree()),
                                                               key=lambda item: int(item[0]))]


In [6]:
# test sources of evidences
source_1, source_2, source_3, source_4, *others = degree__, node_strength_2_deg, betweenness_centrality(G, weight="weight"), closeness_centrality(G)#, eccentricity(G),  approximate_current_flow_betweenness_centrality(G, weight="weight"), current_flow_betweenness_centrality(G),   load_centrality(G),    #eigenvector_centrality(G),
# 1 = degree,  node strength, betweenness, closeness
# print(source_2, source_3)


In [7]:
# [(i,j) for i,j in nx.core_number(gGrahps[14][1]).items()]
# nx.core_number(gGrahps[0][1]).items()
# list(nx.k_core(gGrahps[0][1]))#.items()

In [8]:
# combine sources of evidence into a single dictionary
sources_of_evidences = {
    "Degree": {g[0]: [(i, len(n_neighbor(g[1], i, 1))) for (i, j) in sorted(list(g[1].degree()), key=lambda item: int(item[0]))] for g in gGrahps},
    "Node Strength": {g[0]: [(i, sum(hub_information(g[1], i, 2))) for (i, j) in sorted(list(g[1].degree()), key=lambda item: int(item[0]))] for g in gGrahps},
    "Eccentricity": {g[0]: eccentricity(g[1]) for g in gGrahps},
    "Approx Curr. Flow Betwns Cent'y": {g[0]: approximate_current_flow_betweenness_centrality(g[1]) for g in gGrahps},
    "Betweenness Centrality": {g[0]: betweenness_centrality(g[1], weight="weight") for g in gGrahps},
    # "K Core": {g[0]: k_core((nx.selfloop_edges(g[1]))) for g in gGrahps},
    "Current Flow Betweenness Centrality": {g[0]: current_flow_betweenness_centrality(g[1]) for g in gGrahps},
    "Load Centrality": {g[0]: load_centrality(g[1]) for g in gGrahps}

}
# sources_of_evidences["Degree"]["ABILENE"]


In [9]:
# test case
e_v = 2
varying_examples_multi(source_1, source_2, source_3, source_4,e_v)

[('11', 0.9991533889283354),
 ('1', 0.8102800440940691),
 ('10', 0.43433003311276575),
 ('5', 0.3281087394193372)]

In [10]:
evis = [2,3,4]
selected_nodes_based_on_source ={i:{e: varying_examples_multi(sources_of_evidences["Degree"][i],
sources_of_evidences["Node Strength"][i], 
sources_of_evidences["Eccentricity"][i], 
    sources_of_evidences["Approx Curr. Flow Betwns Cent'y"][i], e) for e in evis} for i, j in gGrahps}
# print(selected_nodes_based_on_source)

selected_nodes_based_on_source_ext = {k2: {k: {x for x, y in sorted(v, key=lambda item: int(item[1]), reverse=True)} for k, v in v2.items()} for k2, v2 in selected_nodes_based_on_source.items()}
# print(selected_nodes_based_on_source_ext)

selected_nodes_based_on_source_ext_len = {k2:{k:len(v) for k,v in v2.items()} for k2,v2 in selected_nodes_based_on_source.items()}


# {i: { "#Nodes":  nx.number_of_nodes(j), "#Edges": nx.number_of_edges(j), "sel_":v} for (i,j), v in zip(gGrahps, selected_nodes_based_on_source.values())}
sel_node_summary={i: {"Graph Name":i, "#Nodes":  nx.number_of_nodes(j), "#Edges": nx.number_of_edges(j), "Diameter":nx.diameter(j), "Density":nx.density(j)}for (i, j)in gGrahps }
# print(sel_node_summary)
sel_to_table = {
    i: (sel_node_summary[i] | selected_nodes_based_on_source_ext[i]) for i, j in gGrahps}
    



In [11]:
sel_node_table = pd.DataFrame.from_dict(
    sel_to_table, orient='index')
# print()
# sel_node_table.to_csv('sel_node_table_w_DSIndex.csv', index=False)


In [12]:
clusters__ = {
    k2: {
        k: cluster_optimal_nodes_test(j, v) for k, v in v2.items()
        }  
        for (i, j), (k2, v2) in zip(
            tqdm(gGrahps), 
            selected_nodes_based_on_source.items()
        ) 
}


100%|██████████| 44/44 [00:05<00:00,  8.23it/s]


In [13]:
# print(clusters__)

In [14]:
# pd.DataFrame.from_dict(clusters__, orient='index')
controller_switch_id = pd.DataFrame.from_records(
    [
        (l1, l2, l3, leaf1)
        for l1, l2_dict in clusters__.items()
        for l2, l3_dict in l2_dict.items()
        for l3, j in l3_dict.items()
        for leaf1 in j
    ],
    columns=
    [
        'Graph Name',
        'No of sources of Evidences',
        'Controller',
        'Allocated Switch'
    ]
)
controller_switch_id.to_csv('controller_switch_id.csv', index=False)


In [15]:
clusters__
# controller_switch_id


{'ABILENE': {2: {'8': {'5', '7'},
   '9': {'0', '2'},
   '10': {'1'},
   '6': {'3', '4'}},
  3: {'7': {'10', '5', '8'}, '1': {'0'}, '2': {'9'}, '6': {'3', '4'}},
  4: {'7': {'1', '10', '3', '6', '8'}, '2': {'0', '9'}, '5': {'4'}}},
 'ABVT': {2: {'11': {'13', '15', '16', '17', '18', '21', '22'},
   '1': {'4', '6', '7', '8', '9'},
   '10': {'14', '19', '2', '20', '3'},
   '5': {'0', '12'}},
  3: {'1': {'12', '15', '22', '4', '5', '6', '8', '9'},
   '21': {'11', '16', '17', '18', '19'},
   '7': {'0'},
   '10': {'13', '14', '2', '20', '3'}},
  4: {'11': {'13', '14', '15', '16', '17', '18', '19', '21', '22'},
   '1': {'4', '6', '8', '9'},
   '0': {'7'},
   '2': {'10', '20', '3'},
   '5': {'12'}}},
 'AGIS': {2: {'9': {'12', '23', '24', '5', '8'},
   '19': {'17', '18', '20', '21', '22', '7'},
   '6': {'1', '4'},
   '10': {'11', '13', '14'},
   '3': {'0', '15', '16', '2'}},
  3: {'5': {'1', '4', '6'},
   '7': {'19', '20', '21'},
   '2': {'22', '23'},
   '9': {'10', '11', '12', '13', '24', '8'}

In [16]:
# save clusters to csv
selected_nodes_optimal = {k3:{k2:{k for k,v in v2.items() }for k2,v2 in v3.items()} for k3,v3 in clusters__.items()}
sel_to_table_optimal = {i: (sel_node_summary[i] | selected_nodes_optimal[i]) for i, j in gGrahps}

sel_node_table_optimal = pd.DataFrame.from_dict(sel_to_table_optimal, orient='index')
# print()
# sel_node_table_optimal.to_csv('sel_node_table_optimal.csv', index=False)


In [17]:
# sel_to_table_optimal
# sel_node_table_optimal


In [18]:
[{x:cluster_optimal_nodes_test(
    gGrahps[0][1], selected_nodes_based_on_source["ABILENE"][x])}for x in [2, 3, 4]]


[{2: {'8': {'5', '7'}, '9': {'0', '2'}, '10': {'1'}, '6': {'3', '4'}}},
 {3: {'7': {'10', '5', '8'}, '1': {'0'}, '2': {'9'}, '6': {'3', '4'}}},
 {4: {'7': {'1', '10', '3', '6', '8'}, '2': {'0', '9'}, '5': {'4'}}}]

In [19]:

clusters__dict = { c: {e: {i: set([i]) | j for i, j in f.items()} for e, f in d.items()} for c, d in clusters__.items()}

# print(clusters__dict)
clusters__pair ={x: {e: {u: k for (k, v) in f.items() for u in v} for e, f in y.items()} for x, y in clusters__dict.items()}
# clusters__pair
# https://networkx.org/documentation/stable/reference/generated/networkx.classes.function.set_node_attributes.html


In [20]:
G = graph_collections[1]
__rand = nx.number_of_nodes(G)
nx.number_of_nodes(G)


23

In [21]:
# [[plot_optimal_cluster(v2,g,k,k2) for k2, v2 in v.items()] for (n,g), (k,v) in zip(tqdm(gGrahps),clusters__pair_small.items())]


In [22]:
def plot_optimal_cluster_test(graph_clusters, g, title, evi, filter_rank=1000):
    
    len_renadom = len(graph_clusters)
    random_colors = color_generator(len_renadom)
    respective_colors = {k:v for  k,v in random_colors.items() if k in set(graph_clusters.values())}
    handles__ = [patches.Patch(color=v, label=k) for k, v in respective_colors.items()]
    graph_clusters = dict(
        sorted(graph_clusters.items(), key=lambda x: int(x[0])))
    # print(len(graph_clusters))
    values = [random_colors.get(v, "#000000")
              for k, v in graph_clusters.items()]
    pos = nx.spring_layout(g)
    nx.draw(g, cmap=plt.get_cmap('viridis'), pos=pos, node_color=values,
            with_labels=True, font_color='white')
    
    title = title.split(".")[0]
    plt.suptitle(f"Graph: {title} \n Sources of Evidences: {evi}, #Nodes: {nx.number_of_nodes(g)}, #Edges: {nx.number_of_edges(g)}, #Controllers: {len(set(graph_clusters.values()))}",
                 fontsize=10, y=0.95)  # f"{title}_{evi}"
    plt.legend(handles=handles__)
    dirpath = Path(f"{Path().absolute()}\images\{title}") if (filter_rank == 1000) else Path(f"{Path().absolute()}\images_{filter_rank}\{title}")
    os.makedirs(dirpath, exist_ok=True) 
    plt.savefig(Path(f"{dirpath}\{title.lower()}_{evi}.png"))
    # plt.show()
    plt.close()
    return "Done"



In [23]:
# [[[plot_optimal_cluster_test(v2, g, k, k2,t) for k2, v2 in v.items()] for (
#     n, g), (k, v) in zip(tqdm(gGrahps), clusters__pair.items())] for t in [2,3,4]]
# [[plot_optimal_cluster_test(v2, g, k, k2) for k2, v2 in v.items()] for (
#     n, g), (k, v) in zip(tqdm(gGrahps), clusters__pair.items())]
os.startfile(r"C:/Users/rotim/Music/Yoruba/SOLA ALLYSON - ÌSÌN.mp3")


In [24]:
# [(node, data) for node, data in G.nodes(data=True)]
node_strength_2_deg = [(i, sum(hub_information(G, i, 2)))
                       for (i, j) in sorted(list(G.degree()), key=lambda item: int(item[0]))]
degree__ = [(i, len(n_neighbor(G, i, 1))) for (i, j) in sorted(list(G.degree()),
                                                               key=lambda item: int(item[0]))]


In [25]:
cluster_optimal_nodes_test(
    gGrahps[1][1], selected_nodes_based_on_source["ABVT"][4])


{'11': {'13', '14', '15', '16', '17', '18', '19', '21', '22'},
 '1': {'4', '6', '8', '9'},
 '0': {'7'},
 '2': {'10', '20', '3'},
 '5': {'12'}}

In [26]:
G = gGrahps[1][1] #graph_collections[1]
sample__ = cluster_optimal_nodes_test(G, selected_nodes_based_on_source["ABVT"][4])
sample__

{'11': {'13', '14', '15', '16', '17', '18', '19', '21', '22'},
 '1': {'4', '6', '8', '9'},
 '0': {'7'},
 '2': {'10', '20', '3'},
 '5': {'12'}}

In [27]:
# new_opti_rank_2
# Get the desired number of controllers from a sorted controller-switch allocation                                                    
sample__filtered = {k: v for k, v in [x for x in sorted(sample__.items(), key=lambda item: len(item[1]), reverse=True)][:3]}
print(sample__)
# factor_set
current_set_result_all = [(i,j) for i,j in sample__filtered.items()]
print(current_set_result_all)
# empty_controllers
empty_controllers = set().union(*[(set([i]).union(j)) for i, j in sample__.items() if i not in sample__filtered.keys()])

# residuals_controllers


{'11': {'21', '13', '22', '17', '14', '15', '18', '19', '16'}, '1': {'9', '8', '6', '4'}, '0': {'7'}, '2': {'10', '3', '20'}, '5': {'12'}}
[('11', {'21', '13', '22', '17', '14', '15', '18', '19', '16'}), ('1', {'9', '8', '6', '4'}), ('2', {'10', '3', '20'})]


In [28]:


cluster_optimal_nodes_test(
    G, selected_nodes_based_on_source["ABVT"][2], is_filtered=True, filter_rank = 5)  #


{'11': {'13', '15', '16', '17', '18', '21', '22'},
 '1': {'4', '6', '7', '8', '9'},
 '10': {'14', '19', '2', '20', '3'},
 '5': {'0', '12'}}

In [66]:

average_case_c_s = {k3: {k2: {k1: {sum(nx.shortest_path_length(j[1], k1, v, weight='weight') for v in v1)/ (nx.number_of_nodes(j[1])-len(v2))} for k1, v1 in v2.items()} for k2, v2 in v3.items()} for j, (k3, v3) in zip(gGrahps, clusters__.items())if k3 == j[0]}
worst_case_c_s = {k3: {k2: {k1: {max(nx.shortest_path_length(j[1], k1, v, weight='weight') for v in v1)} for k1, v1 in v2.items(
)} for k2, v2 in v3.items()} for j, (k3, v3) in zip(gGrahps, clusters__.items())if k3 == j[0]}




In [31]:
average_c_c = {k3: {k2: {k1: set(v2.keys())-set([k1]) for k1 in v2.keys()} for k2, v2 in v3.items()} for k3, v3 in clusters__.items()}
average_case_c_c = {k3: {k2: {k1: {2*sum(nx.shortest_path_length(j[1], k1, v, weight='weight') for v in v1) / (len(v2)**2-len(v2))}
           for k1, v1 in v2.items()} for k2, v2 in v3.items()} for j, (k3, v3) in zip(gGrahps, average_c_c.items())if k3 == j[0]}
worst_case_c_c = {k3: {k2: {k1: {max(nx.shortest_path_length(j[1], k1, v, weight='weight') for v in v1)} for k1, v1 in v2.items(
)} for k2, v2 in v3.items()} for j, (k3, v3) in zip(gGrahps, average_c_c.items())if k3 == j[0]}


In [99]:
# Saving the dataframe as a csv file.

save_cases_as_csv(average_c_c, "Average", "Controller")


Your file has been saved as: average_case_controller_to_controller.csv
